In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass
from sklearn.preprocessing import MinMaxScaler
import math

from operator import itemgetter
import operator

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):


    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(128, f1_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(f2_layer_size, 8),
        nn.Softmax(dim=1),
    )

    return network


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
torch.__version__

'1.12.1'

In [6]:
class DogInput:
    def __init__(self, dogid, raceid,stats, dog,dog_box, lstmCellh,lstmCellc) -> None:
        self.dogid= dogid
        self.raceid = raceid
        self.stats = stats.to('cuda:0')
        self.dog = dog
        self.lstmCellh = lstmCellh.float().to('cuda:0')
        self.lstmCellc = lstmCellc.float().to('cuda:0')
        self.visited = 0
        
        
    def lstm_i(self, lstmInput):
        (self.lstmCellh,self.lstmCellc) = lstmInput
        # self.lstmCellh=self.lstmCellh.to(device)
        # self.lstmCellc=self.lstmCellc.to(device)
        self.visited = self.visited + 1
        # if self.visited>1:
        #     print("FOUND LEAK")
        #     sasdfasd

    def nextrace(self, raceid):
        self.nextrace_id = raceid

    def prevrace(self, raceid):
        self.prevrace_id = raceid

    def lstm_o(self, lstm_o):
        # print(lstm_o[0]._version)
        (lh,lc) = lstm_o
        if self.nextrace_id==-1:
            pass
        else:
            self.dog.races[self.nextrace_id].lstm_i((lh.detach(), lc.detach())) #DETACH
            


In [7]:
class Dog:
    def __init__(self, dogid, hidden_size, layers) -> None:
        self.dogid = dogid
        # self.raceids = raceids #possible dictionary of race id keys dog stat outs
        self.lstmcell = 0
        self.layers = layers
        self.hidden_size = hidden_size
        self.l_debug = None
        self.races = {}

    def add_races(self, raceid, racedate, stats,nextraceid, prevraceid, box):
        self.races[raceid] = DogInput(self.dogid, raceid, stats, self, box, torch.randn(self.hidden_size).clone(),torch.randn(self.hidden_size).clone()) #this is the change
        self.races[raceid].nextrace(nextraceid)
        self.races[raceid].prevrace(prevraceid)

In [8]:
class Race:
    def __init__(self, raceid,trackOHE, dist, classes):
        self.raceid = raceid
        self.race_dist = dist.to('cuda:0')
        self.race_track = trackOHE.to('cuda:0')
        self.classes =  classes.to('cuda:0')

    def add_dogs(self, dogs_list:DogInput):
        self.dog1 = dogs_list[0]
        self.dog2 = dogs_list[1]
        self.dog3 = dogs_list[2]
        self.dog4 = dogs_list[3]
        self.dog5 = dogs_list[4]
        self.dog6 = dogs_list[5]
        self.dog7 = dogs_list[6]
        self.dog8 = dogs_list[7]
        self.dogs = dogs_list

    def nn_input(self):
        input = torch.cat([x.stats for x in self.dogs], dim = 0)
        full_input = torch.cat((self.race_dist,self.race_track, input), dim=0).to(device='cuda:0')
        self.full_input = full_input
        return full_input

    def lstm_input(self):
        
        l_input = [(x.lstmCellh,x.lstmCellc) for x in self.dogs]
        return l_input

    def list_dogs(self):
        dogs_l = [x for x in self.dogs]
        return dogs_l

    def pass_lstm_output(self, lstm_h, lstm_c):
        for i,dog in enumerate(self.dogs):
            
            lh = lstm_h[i]
            lc = lstm_c[i]
            lh,lc = lh.detach(), lc.clone()
            dog.lstm_o((lh,lc))
            dog.dog.l_debug = (lh,lc)
        # zipped_lstm = zip(self.dogs,lstms)
        # [x.lstm_o(y) for x,y in zipped_lstm]

In [9]:
class Races:
    def __init__(self, hidden_size, layers, batch_size = 100) -> None:
        self.racesDict = {}
        self.dogsDict = {}
        self.raceIDs = []
        self.hidden_size = hidden_size
        self.layers = layers
        self.getter = operator.itemgetter(*range(batch_size))

    def add_race(self,raceid:str, trackOHE, dist, classes):
        self.racesDict[raceid] = Race(raceid, trackOHE, dist, classes)
        self.raceIDs.append(raceid)

    def add_dog(self,dogid):
        if dogid not in self.dogsDict.keys():
            self.dogsDict[dogid] = Dog(dogid, self.hidden_size, self.layers)
        else:
            self.dogsDict[dogid] = self.dogsDict[dogid]

    def get_race_input(self, idx) -> Race:
        raceidx = operator.itemgetter(*idx)
        #raceidx  = self.getter(idx)
        race_batch_id = raceidx(self.raceIDs)

        #race_getter = operator.itemgetter(*raceidx)

        races = [self.racesDict[x] for x in race_batch_id] #Returns list of class Race
        

        # raceidx = self.raceIDs[idx]
        #input = torch.cat([x.stats for x in races.dogs.values()], dim = 0)
        #full_input = torch.cat((self.racesDict[raceidx].race_dist,self.racesDict[raceidx].race_track, input), dim=0 )
        # dogs = [x for x in self.racesDict[raceidx].dogs]
        
        return races #self.racesDict[raceidx]

    def get_race_classes(self, idx):
        raceidx = self.raceIDs[idx]
        classes = [x for x in self.raceDict[raceidx].classes]
        return classes


In [10]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.lstm1 = nn.LSTMCell(input_size, hidden_size)
        self.lstm2 = nn.LSTMCell(input_size, hidden_size)
        self.lstm3 = nn.LSTMCell(input_size, hidden_size)
        self.lstm4 = nn.LSTMCell(input_size, hidden_size)
        self.lstm5 = nn.LSTMCell(input_size, hidden_size)
        self.lstm6 = nn.LSTMCell(input_size, hidden_size)
        self.lstm7 = nn.LSTMCell(input_size, hidden_size)
        self.lstm8 = nn.LSTMCell(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size * 8, 64)
        self.fc3 = nn.Linear(64, 8)
        self.hidden_size = hidden_size

    # x represents our data
    def forward(self, race: Race):
        #x = race.nn_input().float().to('cuda:0')
        x = torch.stack([r.full_input.float() for r in race])

        #creates list of LSTM data 
        lstm_ins = [list(i) for i in zip(*[r.lstm_input() for r in race])]

        # creates list of tensors for lstm Cells
        hCell = [torch.stack([x[0] for x in y]) for y in lstm_ins]
        cCell = [torch.stack([x[1] for x in y]) for y in lstm_ins]

        (h1, c1) = self.lstm1(x, (hCell[0], cCell[0]))
        (h2, c2) = self.lstm2(x, (hCell[1], cCell[1]))
        (h3, c3) = self.lstm3(x, (hCell[2], cCell[2]))
        (h4, c4) = self.lstm4(x, (hCell[3], cCell[3]))
        (h5, c5) = self.lstm5(x, (hCell[4], cCell[4]))
        (h6, c6) = self.lstm6(x, (hCell[5], cCell[5]))
        (h7, c7) = self.lstm7(x, (hCell[6], cCell[6]))
        (h8, c8) = self.lstm8(x, (hCell[7], cCell[7]))

        lstm_list = [
            (h1, c1),
            (h2, c2),
            (h3, c3),
            (h4, c4),
            (h5, c5),
            (h6, c6),
            (h7, c7),
            (h8, c8)
        ]

        hCello = [i for i in zip(*[x[0] for x in lstm_list])]
        cCello = [i for i in zip(*[x[1] for x in lstm_list])]

        for i,r in enumerate(race):
            r.pass_lstm_output(hCello[i],cCello[i])
        xhh = torch.cat((h1.clone(),h2.clone(), h3.clone(), h4.clone(), h5.clone(), h6.clone(), h7.clone(), h8.clone()), dim=1).clone()
        xh = self.fc2(xhh)
        xf = self.fc3(xh)

        output = F.softmax(xf, dim=0)
        return output


In [11]:
def build_dataset(data, hidden_size):

    #Load in pickeled dataframe
    resultsdf = pickle.load(data)
    dog_stats_df = pd.DataFrame(resultsdf)
    dog_stats_df = dog_stats_df.fillna(-1).drop_duplicates(subset=['dogid', 'raceid'])
    dog_stats_df['stats_cuda'] = dog_stats_df.apply(lambda x: torch.tensor(x['stats']), axis =1)
    dog_stats_df['box'] = dog_stats_df['stats'].apply(lambda x: x[0])

    #Created RaceDB
    raceDB = Races(hidden_size, 1)

    #Fill in dog portion:

    dog_stats_group = dog_stats_df.sort_values(['date']).groupby(["dogid"])

    for i,j in tqdm(dog_stats_group):
        j["next_race"] = j["raceid"].shift(-1).fillna(-1)
        j["prev_race"] = j["raceid"].shift(1).fillna(-1)
        raceDB.add_dog(i)
        j.apply(lambda x: raceDB.dogsDict[i].add_races(x['raceid'], x['date'], torch.Tensor(x['stats']),x['next_race'], x['prev_race'], x['box']), axis=1)

    #Fill in races portion
    softmin = nn.Softmin()
    races_group = dog_stats_df.groupby(['raceid'])

    null_dog = Dog("nullDog", raceDB.hidden_size, raceDB.layers)
    null_dog_i = DogInput("nullDog", "-1", torch.zeros(16), null_dog,0, torch.zeros(raceDB.hidden_size), torch.zeros(raceDB.hidden_size))
    null_dog_i.nextrace(-1)
    null_dog_i.prevrace(-1)

    null_dog_list = [null_dog] * 8
    #TO FIX LATER PROPER BOX PLACEMENT #FIXED

    races_group = dog_stats_df.groupby(['raceid'])
    for i,j in tqdm(races_group):
    #Track info tensors
        dist = torch.tensor([j.dist.iloc[0]]) 
        trackOHE = torch.tensor(j.trackOHE.iloc[0])
        #margins
        empty_dog_list = [null_dog_i]*8
        empty_margin_list = [100]*8
        boxes_list = [x for x in j['box']]
        margin_list = [x for x in j["place"]]
        dog_list = [raceDB.dogsDict[x].races[i] for x in j["dogid"]]

        #adjustedMargin = [margin_list[x-1] for x in boxes_list]
        for n,x in enumerate(boxes_list):
            empty_margin_list[x-1] = margin_list[n]
            empty_dog_list[x-1] = dog_list[n]
        adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
        #adjusted_dog_list = [dog_list[x-1] for x in boxes_list]
        
        raceDB.add_race(i,trackOHE,dist, adjustedMargin)
        
        
        # List of Dog Input??
        raceDB.racesDict[i].add_dogs(empty_dog_list)
        raceDB.racesDict[i].nn_input()


    return raceDB



In [12]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
dog_stats_file = open( 'dog_stats_df.npy', 'rb')
raceDB = build_dataset(dog_stats_file, 5)

  0%|          | 0/72073 [00:00<?, ?it/s]C:\Users\Nick\AppData\Local\Temp\ipykernel_15288\1524283141.py:51: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  adjustedMargin = softmin(torch.tensor(empty_margin_list)).to('cuda:0')
100%|██████████| 72073/72073 [00:57<00:00, 1255.59it/s]


In [13]:
newnet = Net(144,5)
loss_function = nn.MSELoss()
optimizer = optim.Adam(newnet.parameters(), lr=0.001)
newnet = newnet.to('cuda:0')
newnet.forward

<bound method Net.forward of Net(
  (lstm1): LSTMCell(144, 5)
  (lstm2): LSTMCell(144, 5)
  (lstm3): LSTMCell(144, 5)
  (lstm4): LSTMCell(144, 5)
  (lstm5): LSTMCell(144, 5)
  (lstm6): LSTMCell(144, 5)
  (lstm7): LSTMCell(144, 5)
  (lstm8): LSTMCell(144, 5)
  (fc2): Linear(in_features=40, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
)>

In [14]:
#Testing
def validate_model(model, batch_size):
    torch.autograd.set_detect_anomaly(True)
    list_t = [] 
    last = 0
    loss_val = 0 
    correct = 0
    total = 0
    with torch.no_grad():
        for i in trange(60000,70000,batch_size):   
            races_idx = range(last,last+batch_size)
            last = i
            race = raceDB.get_race_input(races_idx)
            X = race
            y = torch.stack([x.classes for x in race])
            output = model(X)
            #print(y)
            _, actual = torch.max(y.data, 1)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            #print(actual)
            correct += (predicted == actual).sum().item()
            total +=10



            #loss = loss_function(output, y)
            #optimizer.zero_grad()
            #newnet.zero_grad()
            #loss.backward(retain_graph=True)  
            #optimizer.step()
            #if i %5000 == 0:
            #    print(loss)
        # optimizer.step() 
        #print(loss)
    print(f"accuray: {correct/total}")

In [21]:
torch.autograd.set_detect_anomaly(True)
list_t = [] 
last = 0
batch_size = 100
for epoch in range(200): 
    for i in trange(0,60000,batch_size):   
        races_idx = range(last,last+batch_size)
        last = i
        race = raceDB.get_race_input(races_idx)
        X = race
        # print(X)
        #y = race.classes.to(device)
        y = torch.stack([x.classes for x in race])
        #print(y)
        output = newnet(X)
        # print(y)
        # print(output)
        #
        #print(output) 
        #print([x.dogid for x in X.dogs])
        loss = loss_function(output, y)
        optimizer.zero_grad()
        #newnet.zero_grad()
        loss.backward(retain_graph=True)  
        optimizer.step()
        # if i %5000 == 0:
        #     #print(loss)
    # optimizer.step() 
    print(loss)
    validate_model(newnet, 8)

  2%|▏         | 30/1250 [00:00<00:04, 299.93it/s]

tensor(0.0551, device='cuda:0', grad_fn=<MseLossBackward0>)


  0%|          | 0/600 [00:00<?, ?it/s]

accuray: 0.18696


  4%|▍         | 54/1250 [00:00<00:04, 272.03it/s]

...

  2%|▏         | 23/1250 [00:00<00:05, 221.10it/s]

tensor(0.0552, device='cuda:0', grad_fn=<MseLossBackward0>)


  0%|          | 0/600 [00:00<?, ?it/s]

accuray: 0.18936


 21%|██▏       | 128/600 [01:14<04:27,  1.76it/s]

In [20]:
print("YYY")

YYY


In [ ]:
dlstm = [list(i) for i in zip(*[x.lstm_input() for x in test_data])]
pprint.pprint(dlstm[0])

[(tensor([-0.8675, -0.8717, -1.5944, -0.5800, -0.0219], device='cuda:0'),
  tensor([-0.5273,  0.3271,  0.7316,  0.4938,  1.6515], device='cuda:0')),
 (tensor([ 1.8622,  0.6056, -0.5197, -0.5091,  0.7088], device='cuda:0'),
  tensor([-1.1639, -0.4896,  0.9631, -0.3343, -0.7571], device='cuda:0')),
 (tensor([ 1.0674, -2.0852, -1.0210, -0.0981, -0.2083], device='cuda:0'),
  tensor([-1.3528,  1.0761,  1.2515,  0.3496,  0.6863], device='cuda:0'))]


In [ ]:
cCell = [torch.stack([x[0] for x in y]) for y in dlstm]
hCell = [torch.stack([x[1] for x in y]) for y in dlstm]

In [ ]:
type(cCell[0])

torch.Tensor

In [ ]:
hCell

[tensor([[-0.5273,  0.3271,  0.7316,  0.4938,  1.6515],
         [-1.1639, -0.4896,  0.9631, -0.3343, -0.7571],
         [-1.3528,  1.0761,  1.2515,  0.3496,  0.6863]], device='cuda:0'),
 tensor([[-0.1061, -0.2952,  1.9279,  0.9492, -0.2213],
         [ 0.7067, -1.4521, -0.4989, -0.0936,  0.2086],
         [-1.3173, -0.1100, -0.9247, -0.7827, -0.1964]], device='cuda:0'),
 tensor([[-0.6996, -0.1256, -0.8835, -0.0646, -2.0322],
         [ 1.0575, -0.2605, -0.1528, -0.4997,  0.4953],
         [ 0.8972,  0.1420,  1.7114,  0.2030,  0.3664]], device='cuda:0'),
 tensor([[ 0.9384, -1.7895, -0.1702,  0.8365, -0.3397],
         [ 0.9431,  0.7915, -0.1227, -0.3582, -1.5961],
         [-0.5381,  1.8346,  1.1186, -0.0132, -1.0070]], device='cuda:0'),
 tensor([[-1.3374,  0.7250,  0.9002,  1.6969, -0.3953],
         [-0.1624, -1.7428,  0.5430, -1.1477,  1.3154],
         [-2.4506, -0.3516, -0.2425, -1.9107, -1.3467]], device='cuda:0'),
 tensor([[-0.0748, -1.1441, -0.9800, -0.3879,  1.4227],
         